In [13]:
# 不要Warnings
import warnings
warnings.filterwarnings('ignore')

# 导入所需的库
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import pandas as pd

train_data = pd.read_csv('./dataset/permissions.csv')
labels = train_data["Class"]
data = train_data.iloc[:,1:]
data = data.iloc[:,:].values
srf = RF(n_estimators=500, n_jobs=-1)
clf_s = cross_val_score(srf, data, labels, cv=10)
print(clf_s)

# 绘制混淆矩阵
srf.fit(data, labels)
cm = confusion_matrix(labels, srf.predict(data))
print(cm)



[0.66666667 0.55555556 0.66666667 0.77777778 0.66666667 1.
 0.66666667 0.77777778 0.77777778 0.55555556]
[[22  0  0  0]
 [ 0 22  0  0]
 [ 0  0  9  0]
 [ 0  0  1 36]]


In [16]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.preprocessing.text as T
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

train_data = pd.read_csv('D:\\android\\dataset\\permissions.csv')
labels = train_data["Class"]
data = train_data.iloc[:,1:]
train_data = data.iloc[:,:].values
from sklearn.model_selection import StratifiedKFold
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(train_data, labels):
    model = keras.Sequential()
    model.add(layers.Dense(50,input_dim = 51, activation = 'relu'))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=['acc']
    )
    model.fit(train_data[train],labels[train],epochs=60, batch_size=256,verbose = 0)
    scores = model.evaluate(train_data[test], labels[test], verbose=0)
    print(scores[1])
    cvscores.append(scores[1])
print(cvscores)


ModuleNotFoundError: No module named 'tensorflow'